In [76]:
import logging
import boto3
from botocore.exceptions import ClientError
import os


def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    s3_client = boto3.client('s3')
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True

In [86]:
#Parameters
prefix = "filipp"
filename_to_upload = "test.txt"
bucket_name = "module1iot"

In [56]:
#Upload of a single txt.file
s3 = boto3.client('s3')
with open(filename_to_upload, "rb") as f:
    s3.upload_fileobj(f, bucket_name, prefix + "/" + filename_to_upload)

In [69]:
s3 = boto3.resource('s3')
s3.meta.client.upload_file("test.txt", 'module1iot', 'filipp/meta.txt')

In [88]:
#Delete the single object
delete_what = "meta.txt"#^filename_to_upload
s3 = boto3.client('s3')
s3.delete_object(Bucket=bucket_name, Key= prefix + "/" + delete_what)

{'ResponseMetadata': {'RequestId': '03HXCVADDGMK0PEE',
  'HostId': 'rvR2O6KamtW7x4r1zeA0+5xaTzCIWkKTmUJ6zgPnQ0Cq35uTfjGa9kqrL7oCMzsq914KbMM9EVM=',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'rvR2O6KamtW7x4r1zeA0+5xaTzCIWkKTmUJ6zgPnQ0Cq35uTfjGa9kqrL7oCMzsq914KbMM9EVM=',
   'x-amz-request-id': '03HXCVADDGMK0PEE',
   'date': 'Wed, 17 Aug 2022 09:08:42 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}

In [89]:
#Read Files in Bucket only for the prefix
import boto3
session = boto3.Session()
s3 = session.resource('s3')
client = boto3.client('s3')

my_bucket = s3.Bucket(bucket_name)
for my_bucket_object in my_bucket.objects.all():
    if(my_bucket_object.key.split("/")[0]=="filipp"):
        print(my_bucket_object.key)
    #print(my_bucket_object.key)
        #metadata = client.head_object(
        #    Bucket= bucket_name,
        #    Key= my_bucket_object.key,
        #)
        #print(metadata)

filipp/img235235.json
filipp/test.txt


In [94]:
import json
from datetime import datetime, timezone


def create_json(object_key, event_time, qr_code):
    # Json skeleton
    json_skeleton = {
        "pictureFileName": object_key,
        "timestamp": event_time,
        "qrCode": qr_code
    }
    #print("# # # JSON-Metafile: ", json_skeleton)
    # Convert to JSON
    return json.dumps(json_skeleton)

img = "123.png"
actual_json = create_json(prefix + "/" + img, str(datetime.now(timezone.utc)) , "oufhosd8f0f0ds")

In [39]:
json_file_name = "img235235.json"
content = actual_json
s3.Object(bucket_name, prefix + "/" + json_file_name).put(Body=content)

{'ResponseMetadata': {'RequestId': 'DB9A13T444AM3V1N',
  'HostId': 'Onn+Mofx/nRkodKFDprTIulxfLFgb2NVkW4LBzSg6I9cqptYuNqgx74d/11k7mgVSnvxOzSCP3M=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'Onn+Mofx/nRkodKFDprTIulxfLFgb2NVkW4LBzSg6I9cqptYuNqgx74d/11k7mgVSnvxOzSCP3M=',
   'x-amz-request-id': 'DB9A13T444AM3V1N',
   'date': 'Wed, 17 Aug 2022 08:05:52 GMT',
   'etag': '"c991cda2370e356a7800f51b147177d1"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c991cda2370e356a7800f51b147177d1"'}

In [96]:
#aws configure required, see here: https://boto3.amazonaws.com/v1/documentation/api/latest/guide/credentials.html

import os
import boto3
session = boto3.session.Session()
s3_client = session.client('s3')

# Method 1: Object.put()
def push_to_s3(filename):
    #use profile which was created with aws configure --profile awacademys3user
    session = boto3.session.Session()
    s3 = session.resource('s3')
    object = s3.Object('module1iot', filename)
    object.put(Body=filename)

def upload_file(file_name, bucket, object_name=None):
    """Upload a file to an S3 bucket

    :param file_name: File to upload
    :param bucket: Bucket to upload to
    :param object_name: S3 object name. If not specified then file_name is used
    :return: True if file was uploaded, else False
    """

    # If S3 object_name was not specified, use file_name
    if object_name is None:
        object_name = os.path.basename(file_name)

    # Upload the file
    try:
        response = s3_client.upload_file(file_name, bucket, object_name)
    except ClientError as e:
        logging.error(e)
        return False
    return True
    
if __name__ == "__main__":
    path = "images/"
    identifier = "filipp/"
    for filename in os.listdir(path):
        if filename.endswith(".png"): 
            print("Start pushing file " + filename + "to S3")
            with open(path + filename, "rb") as f:
                s3_client.upload_fileobj(f, "module1iot", identifier + str(filename))
                s3.Object(bucket_name, prefix + "/" + filename.split(".")[0] + ".json").put(Body=create_json(prefix + "/" + filename, str(datetime.now(timezone.utc)) , "i was too lazy"))
            print("Finished")
            continue
        else:
            continue

Start pushing file 89779D1A6581.pngto S3
Finished
Start pushing file 3F630C84A097.pngto S3
Finished
Start pushing file 5B124CDBB2CF.pngto S3
Finished
Start pushing file A0CE582262F8.pngto S3
Finished
Start pushing file 039BD2480D54.pngto S3
Finished
Start pushing file 132B6D657FC4.pngto S3
Finished
Start pushing file 355F36B79EEF.pngto S3
Finished
Start pushing file 386F626832BC.pngto S3
Finished
Start pushing file C6688D71F7CC.pngto S3
Finished
Start pushing file 92D42004DC9A.pngto S3
Finished
Start pushing file B68B30C88E9C.pngto S3
Finished
Start pushing file E98891DFF18E.pngto S3
Finished
Start pushing file 76F0ACD35643.pngto S3
Finished
Start pushing file 74C52EBE7FF4.pngto S3
Finished
Start pushing file 5F37783C7829.pngto S3
Finished
Start pushing file 52F4FBC55FF7.pngto S3
Finished
Start pushing file 191E0E80E332.pngto S3
Finished
Start pushing file D99FE5C7D40C.pngto S3
Finished
Start pushing file 96DD29B4BE26.pngto S3
Finished
Start pushing file 13BDB60CD560.pngto S3
Finished
